In [1]:
from robust_mean import huberReg

In [2]:
import numpy as np
import math
from sklearn.linear_model import HuberRegressor
import numpy.random as rgt
# ... other necessary imports ...

# Assuming your class definition `huberReg` is already provided above

In [98]:
# Generate some sample data
n, d = 5000, 5  # Adjust n and d according to your needs
X = np.random.randn(n, d)  # Random data for X
truebeta = rgt.multivariate_normal(np.zeros(d), np.identity(d))
Y = X.dot(truebeta) + np.random.randn(n)  # Linear relationship with noise

# Create an instance of your class
huber_regression = huberReg(X, Y)

# Set initial parameters for noisy Huber regression
beta0 = np.zeros(d)
epsilon = 0.1
T = 10
delta = 0.01
eta = 1
row_norms = np.linalg.norm(X, axis=1)

# Finding the maximum 2-norm among all rows
gamma = np.max(row_norms)
gamma

4.8129523330015545

In [101]:
# Run the noisy Huber regression
noisy_huber_result = huber_regression.noisy_huber_reg_lowdim(beta0=beta0, epsilon=epsilon, T=T, delta=delta, eta=0.001, gamma=2)
print("Noisy Huber Regression Result:", noisy_huber_result[0])

# Ordinary Huber Regression for comparison
huber_reg = HuberRegressor().fit(X, Y)
ordinary_huber_coefficients = huber_reg.coef_
print("Ordinary Huber Regression Coefficients:", ordinary_huber_coefficients)

# Compare the results (You can use metrics like MSE, coefficients, predictions, etc.)

Noisy Huber Regression Result: [-0.29956248  0.36527583  0.36741373 -0.77468552  0.57255891]
Ordinary Huber Regression Coefficients: [ 1.60519073  0.80633407  0.56950276  0.5108081  -0.07697369]


In [102]:
truebeta

array([ 1.61244219,  0.78881411,  0.56736909,  0.5271901 , -0.0641991 ])

In [ ]:
noisy_huber_result[2]

In [ ]:
noisy_huber_result[1][1]